# MODPATH example

In [1]:
import sys
import shutil
import os
import glob
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# run installed version of flopy or add local path
try:
    import flopy
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    import flopy

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('pandas version: {}'.format(pd.__version__))
print('flopy version: {}'.format(flopy.__version__))

if not os.path.exists("data"):
    os.mkdir("data")

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.19.2
matplotlib version: 3.3.4
pandas version: 1.2.3
flopy version: 3.3.3


In [3]:
moddirmp=r"C:\Users\Quilson2\OneDrive\Desktop\modpath_7_2_001\modpath_7_2_001\bin\mpath7"

In [ ]:
model_ws = r'C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\hws-PortilloD\Assignment\HW8 ReverseMidterm'
m = flopy.modflow.Modflow.load('TestHW8.nam', model_ws=model_ws)
m.get_package_list()

In [ ]:
mffiles = glob.glob(os.path.join('..', 'data', 'mp6', 'TestHW8'))
for f in mffiles:
    print(f)
    shutil.copy(f, os.path.join('data', os.path.split(f)[-1]))

In [ ]:

model_ws = r'C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\hws-PortilloD\Assignment\HW8 ReverseMidterm'
m = flopy.modflow.Modflow.load('TestHW8.nam', model_ws=model_ws)
m.get_package_list()

In [ ]:

nrow, ncol, nlay, nper = m.nrow_ncol_nlay_nper
nrow, ncol, nlay, nper

In [ ]:

m.dis.steady.array

In [ ]:
m.write_input()

In [ ]:
hdsfile = flopy.utils.HeadFile(os.path.join(model_ws,'TestHW8.HDS'))
hdsfile.get_kstpkper()

In [ ]:
hds = hdsfile.get_data(kstpkper=(0, 200))

In [ ]:
plt.imshow(hds[0, :, :])
plt.colorbar();

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mapview = flopy.plot.PlotMapView(model=m, layer=1)
quadmesh = mapview.plot_ibound()
linecollection = mapview.plot_grid()
#riv = mapview.plot_bc('RIV', color='g', plotAll=True)
quadmesh = mapview.plot_bc('WEL', kper=1, plotAll=True)
contour_set = mapview.contour_array(hds, 
                                    levels=np.arange(np.min(hds),np.max(hds),2.0), colors='b')
plt.clabel(contour_set, inline=1, fontsize=14);

In [ ]:
mp = flopy.modpath.Modpath6(modelname='Testparticles',
                            exe_name='mp6',
                            modflowmodel=m,
                            model_ws=r'C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\hws-PortilloD\Assignment\HW8 ReverseMidterm',
                            dis_file=m.name+'.dis',
                            head_file=m.name+'.hds',
                            budget_file=m.name+'.bas')

mpb = flopy.modpath.Modpath6Bas(mp, hdry=m.lpf.hdry, laytyp=m.lpf.laytyp)

# start the particles at begining of per 10, step 1, as in example 3 in MODPATH6 manual
# (otherwise particles will all go to river)
sim = mp.create_mpsim(trackdir='forward', simtype='pathline', packages='RCH', start_time=(0, 0, 1.))
mp.write_input()

mp.run_model(silent=False)

In [ ]:
fpth = os.path.join(r'C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\hws-PortilloD\Assignment\HW8 ReverseMidterm','Testparticles.mpend')
#fpth = os.path.join(r'C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\hws-PortilloD\Assignment\HW8 ReverseMidterm','ex6.mpend')
epobj = flopy.utils.EndpointFile(fpth)
well_epd = epobj.get_destination_endpoint_data(dest_cells=[(2, 200, 200)]) 
# returns record array of same form as epobj.get_all_data()

In [ ]:
well_epd[0:200]

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mapview = flopy.plot.PlotMapView(model=m, layer=2)
quadmesh = mapview.plot_ibound()
linecollection = mapview.plot_grid()
#riv = mapview.plot_bc('RIV', color='g', plotAll=True)
quadmesh = mapview.plot_bc('WEL', kper=1, plotAll=True)
contour_set = mapview.contour_array(hds, 
                                     levels=np.arange(np.min(hds),np.max(hds),2.0), colors='b')
plt.clabel(contour_set, inline=1, fontsize=14)
mapview.plot_endpoint(well_epd, direction='starting', colorbar=True);

In [ ]:
fpth = os.path.join(r'C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\hws-PortilloD\Assignment\HW8 ReverseMidterm','starting_locs.shp')
print(type(fpth))
epobj.write_shapefile(well_epd, direction='starting', shpname=fpth, mg=m.modelgrid)

In [ ]:
# make a selection of cells that terminate in the well cell = (4, 12, 12)
pthobj = flopy.utils.PathlineFile(os.path.join(r'C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\hws-PortilloD\Assignment\HW8 ReverseMidterm','ex6.mppth'))
well_pathlines = pthobj.get_destination_pathline_data(dest_cells=[(1, 20, 20)])

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mapview = flopy.plot.PlotMapView(model=m, layer=2)
quadmesh = mapview.plot_ibound()
linecollection = mapview.plot_grid()
#riv = mapview.plot_bc('RIV', color='g', plotAll=True)
quadmesh = mapview.plot_bc('WEL', kper=1, plotAll=True)
contour_set = mapview.contour_array(hds, 
                                     levels=np.arange(np.min(hds),np.max(hds),1.0), colors='b')
plt.clabel(contour_set, inline=1, fontsize=14)

mapview.plot_endpoint(well_epd, direction='starting', colorbar=True)
#for now, each particle must be plotted individually 
#(plot_pathline() will plot a single line for recarray with multiple particles)
#for pid in np.unique(well_pathlines.particleid):
#   modelmap.plot_pathline(pthobj.get_data(pid), layer='all', colors='red');
mapview.plot_pathline(well_pathlines, layer='all', colors='red');

In [ ]:
# one line feature per particle
pthobj.write_shapefile(well_pathlines,
                       direction='starting', shpname=os.path.join('data','pathlines.shp'),
                       mg=m.modelgrid)

# one line feature for each row in pathline file 
# (can be used to color lines by time or layer in a GIS)
pthobj.write_shapefile(well_pathlines, one_per_particle=False, shpname=os.path.join('data','pathlines_1per.shp'),
                       mg=m.modelgrid)

In [ ]:
model_ws = r'C:\Users\Quilson2\OneDrive\Documents\SPRING21\582-GWMod\hws-PortilloD\Assignment\HW8 ReverseMidterm'
m2 = flopy.modflow.Modflow.load('TestHW8.nam', model_ws=model_ws, exe_name='mf2005')
m2.get_package_list()

In [ ]:

m2.nrow_ncol_nlay_nper

In [ ]:
m2.wel.stress_period_data.data

In [ ]:
node_data = np.array([(3, 12, 12, 'well1', 'skin', -1, 0, 0, 0, 1., 2., 5., 6.2),
                      (4, 12, 12, 'well1', 'skin', -1, 0, 0, 0, 0.5, 2., 5., 6.2)], 
                     dtype=[('k', int), ('i', int), ('j', int), 
                            ('wellid', object), ('losstype', object), 
                            ('pumploc', int), ('qlimit', int), 
                            ('ppflag', int), ('pumpcap', int), 
                            ('rw', float), ('rskin', float), 
                            ('kskin', float), ('zpump', float)]).view(np.recarray)

stress_period_data = {0: np.array([(0, 'well1', -150000.0)], dtype=[('per', int), ('wellid', object), 
                                                            ('qdes', float)])}

In [ ]:
m2.name = 'Example_mnw'
m2.remove_package('WEL')
mnw2 = flopy.modflow.ModflowMnw2(model=m2, mnwmax=1,
                                 node_data=node_data, 
                                 stress_period_data=stress_period_data, 
                                 itmp=[1, -1, -1])
m2.get_package_list()

In [ ]:

m2.write_input()

m2.run_model(silent=False)

In [ ]:

mp = flopy.modpath.Modpath6(modelname='ex6mnw',
                            exe_name='mp6',
                            modflowmodel=m2,
                            model_ws='data',
                            dis_file=m.name+'.DIS',
                            head_file=m.name+'.hds',
                            budget_file=m.name+'.cbc')

mpb = flopy.modpath.Modpath6Bas(mp, hdry=m2.lpf.hdry, laytyp=m2.lpf.laytyp, ibound=1, prsity=0.1)
sim = mp.create_mpsim(trackdir='backward', simtype='pathline', packages='MNW2')

mp.write_input()

mp.run_model(silent=False)

In [ ]:
pthobj = flopy.utils.PathlineFile(os.path.join('data','ex6mnw.mppth'))
epdobj = flopy.utils.EndpointFile(os.path.join('data','ex6mnw.mpend'))
well_epd = epdobj.get_alldata()
well_pathlines = pthobj.get_alldata() # returns a list of recarrays; one per pathline

In [ ]:
fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(1, 1, 1, aspect='equal')
mapview = flopy.plot.PlotMapView(model=m2, layer=2)
quadmesh = mapview.plot_ibound()
linecollection = mapview.plot_grid()
riv = mapview.plot_bc('RIV', color='g', plotAll=True)
quadmesh = mapview.plot_bc('MNW2', kper=1, plotAll=True)
contour_set = mapview.contour_array(hds, 
                                     levels=np.arange(np.min(hds),np.max(hds),0.5), colors='b')
plt.clabel(contour_set, inline=1, fontsize=14)

mapview.plot_pathline(well_pathlines, travel_time='<10000',
                       layer='all', colors='red');